In [9]:
from stmpy import Machine, Driver

import ipywidgets as widgets
from IPython.display import display

class Charger:
    
    def on_buttonConnect_press(self, b):
        self.stm.send('car_connected') # <---- here we send a signal
    
    def on_buttonCharge_press(self, b):
        self.stm.send('request_charging') # <---- here we send a signal
    
    def on_buttonDisconnect_press(self, b):
        self.stm.send('car_disconnected') # <---- here we send a signal

    def on_buttonChargingFinished_press(self, b):
        self.stm.send('charging_finished') # <---- here we send a signal

    def on_buttonChargingAborted_press(self, b):
        self.stm.send('charging_aborted') # <---- here we send a signal

    def on_init(self):
        print("Charger created \n")
        # load images and store them
        self.buttonConnect = widgets.Button(description="Connect car")
        self.buttonConnect.on_click(self.on_buttonConnect_press)
        
        self.buttonCharge = widgets.Button(description="Charge")
        self.buttonCharge.on_click(self.on_buttonCharge_press)

        self.buttonDisconnect = widgets.Button(description="Disconnect car")
        self.buttonDisconnect.on_click(self.on_buttonDisconnect_press)

        self.buttonChargingFinished = widgets.Button(description="Finish charging")
        self.buttonChargingFinished.on_click(self.on_buttonChargingFinished_press)

        self.buttonChargingAborted = widgets.Button(description="Abort charging")
        self.buttonChargingAborted.on_click(self.on_buttonChargingAborted_press)

        display(self.buttonConnect, self.buttonCharge, self.buttonDisconnect, self.buttonChargingFinished, self.buttonChargingAborted)

    def send_mqtt_connected(self):
        print("Connected")
        self.mqtt_client.publish("from_charger", "Car is connected to charger")
    
    def send_mqtt_disconnected(self):
        print("Disconnected")
        self.mqtt_client.publish("from_charger", "Car is disconnected from charger")

    def send_mqtt_charging(self):
        print("Charging")
        self.mqtt_client.publish("from_charger", "Car is charging")
    
    def send_mqtt_finished(self):
        print("Finished")
        self.mqtt_client.publish("from_charger", "Charging is finished")
    
    def send_mqtt_aborted(self):
        print("Aborted")
        self.mqtt_client.publish("from_charger", "Charging is aborted")


In [10]:
t0_initial = {'source': 'initial',
      'target': 'idle',
      'effect': 'on_init'}

t1_car_connected = {'trigger': 'car_connected',
        'source': 'idle',
        'target': 'connected',
        'effect': 'send_mqtt_connected'
}

t2_request_charging = {'trigger': 'request_charging',
        'source': 'connected',
        'target': 'charging',
        'effect': 'send_mqtt_charging'
}
t3_car_disconnected = {'trigger': 'car_disconnected',
        'source': 'connected',
        'target': 'idle',
        'effect': 'send_mqtt_disconnected'
}
t4_charging_finished = {'trigger': 'charging_finished',
        'source': 'charging',
        'target': 'connected',
        'effect': 'send_mqtt_finished'
}
t5_charging_aborted = {'trigger': 'charging_aborted',
        'source': 'charging',
        'target': 'connected',
        'effect': 'send_mqtt_aborted'
}

idle = {'name': 'idle'}

connected = {'name': 'connected'}

charging = {'name': 'charging'}


In [27]:
from threading import Thread

import paho.mqtt.client as mqtt

broker, port = 'test.mosquitto.org', 1883

class MQTT_CHARGER:
    def __init__(self):
        self.client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message

    def on_connect(self, client, userdata, flags, rc):
        print("on_connect(): {}".format(mqtt.connack_string(rc)))
        print("connection hard code debug print")
    
    def on_message(self, client, userdata, message):
        print("on_message(): topic: '{}', message: '{}'".format(message.topic, message.payload.decode()))
        if message.payload.decode() == "Car is charging":
            print("Car is charging, bitch!")
        elif message.payload.decode() == "Car is connected to charger":
            print("Charger message recieved")
        # elif more stuff

    def start(self, broker, port):
        
        print("Connecting to broker:", broker, "port:", port)
        self.client.connect(broker, port)

        self.client.subscribe('from_charger')
        
        try:
            # line below should not have the () after the function!
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()

In [28]:
charger = Charger()
machine = Machine(name='charger', transitions=[t0_initial, t1_car_connected, 
                                               t2_request_charging, t3_car_disconnected, 
                                               t4_charging_finished, t5_charging_aborted], 
                                               states = [idle, connected, charging], obj=charger)

charger.stm = machine

driver = Driver()
driver.add_machine(machine)

mqtt_charger = MQTT_CHARGER()
charger.mqtt_client = mqtt_charger.client
mqtt_charger.stm_driver = driver

driver.start()
mqtt_charger.start(broker, port)


Charger created 
Connecting to broker: test.mosquitto.org port: 1883



C:\Users\Andre\AppData\Local\Temp\ipykernel_17756\2827356388.py:9: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  self.client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)


Button(description='Connect car', style=ButtonStyle())

Button(description='Charge', style=ButtonStyle())

Button(description='Disconnect car', style=ButtonStyle())

Button(description='Finish charging', style=ButtonStyle())

Button(description='Abort charging', style=ButtonStyle())

on_connect(): Connection Accepted.
connection hard code debug print
Connected
on_message(): topic: 'from_charger', message: 'Car is connected to charger'
Disconnected
on_message(): topic: 'from_charger', message: 'Car is disconnected from charger'
Connected
on_message(): topic: 'from_charger', message: 'Car is connected to charger'
on_message(): topic: 'from_charger', message: 'Car is connected to charger'
on_message(): topic: 'from_charger', message: 'Car is charging'
Car is charging, bitch!
on_message(): topic: 'from_charger', message: 'Charging is finished'
on_message(): topic: 'from_charger', message: 'Car is charging'
Car is charging, bitch!
on_message(): topic: 'from_charger', message: 'Charging is finished'
on_message(): topic: 'from_charger', message: 'Car is charging'
Car is charging, bitch!
on_message(): topic: 'from_charger', message: 'Charging is finished'
on_message(): topic: 'from_charger', message: 'Car is charging'
Car is charging, bitch!
on_message(): topic: 'from_char